# Preliminaries

In [400]:
import pandas as pd
import numpy as np
import pathlib
from ipywidgets import interact

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import layout, row, column
from bokeh.models import ColumnDataSource, Plot, Range1d, Panel, Tabs, FactorRange
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import Div

output_notebook()

# Define paths.
PATH_DATA = pathlib.Path(r"../data")

df_stats = pd.read_csv(PATH_DATA/'df_stats.csv', index_col=0)
df_sorted = df_stats.sort_values('votes', ascending=True) # Sort by popularity.
#cds_stats = ColumnDataSource(df_stats)

df_raw = pd.read_excel(PATH_DATA/'responses.xlsx', sheet_name='Form Responses 1')
df_raw.rename(columns={'Timestamp':'timestamp', 'What is your favourite Pokémon?':'vote'}, inplace=True)
df_raw.dropna(inplace=True) # Remove any potential NaN.

#############
df_time = df_raw.query('vote=="Bulbasaur"')
df_time = df_time.groupby(pd.Grouper(key='timestamp', freq='1h')).count()
df_time['timestamp'] = df_time.index

x_times = df_time['timestamp'].dt.strftime('%H:%M')

############


x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

# Create Pokemon sprite.
sprite = Div(text="""<img src="https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png" alt="sprite">""", width=150, height=150)


# Create the "Overall" plot.
source_overall = ColumnDataSource(df_sorted[['name', 'votes', 'generation']])
pokemon_names = source_overall.data['name']
pokemon_votes = source_overall.data['votes']

# Notice that initializing the figure with y_range=pokemon_names 
# doesn't allow the option to bound the plot.
p_overall = figure(y_range=FactorRange(factors=pokemon_names, bounds=(0, len(pokemon_names))), 
                   x_axis_label='Votes', plot_height=250)
p_overall.hbar(y='name', left=0, right='votes', height=0.9, source=source_overall)
p_overall.x_range = Range1d(0, max(pokemon_votes)*1.05, bounds=(0, max(pokemon_votes)*1.05))
p_overall.ygrid.grid_line_color = None

hover_overall = HoverTool()
hover_overall.tooltips = """
    <div>
        <h3>@name</h3>
        <div><strong>Generation: </strong>@generation</div>
        <div><strong>Votes: </strong>@votes</div>
    </div>
"""
p_overall.add_tools(hover_overall)
    
    
# Create the "Generation" plot.
generation=1
df_generation = df_sorted.query('generation==' + str(generation))
source_generation = ColumnDataSource(df_generation[['name', 'votes']])
pokemon_names_gen = source_generation.data['name']
pokemon_votes_gen = source_generation.data['votes']

p_generation = figure(y_range=FactorRange(factors=pokemon_names_gen, bounds=(0, len(pokemon_names_gen))), 
                      x_axis_label='Votes', plot_height=250)
r_generation = p_generation.hbar(y='name', left=0, right='votes', height=0.9, source=source_generation)
p_generation.x_range = Range1d(0, max(pokemon_votes_gen)*1.05, bounds=(0, max(pokemon_votes_gen)*1.05))
p_generation.ygrid.grid_line_color = None

hover_generation = HoverTool()
hover_generation.tooltips = """
    <div>
        <h3>@name</h3>
        <div><strong>Votes: </strong>@votes</div>
    </div>
"""
p_generation.add_tools(hover_generation)


# Create the "Family" plot.
df_families = df_stats[['votes', 'family']].groupby(['family']).sum().sort_values('votes', ascending=True)
source_family = ColumnDataSource(df_families)
pokemon_names_fam = source_family.data['family']
pokemon_votes_fam = source_family.data['votes']

p_family = figure(y_range=FactorRange(factors=pokemon_names_fam, bounds=(0, len(pokemon_names_fam))), 
                  x_axis_label='Votes', plot_height=250)
p_family.hbar(y='family', left=0, right='votes', height=0.9, source=source_family)
p_family.x_range = Range1d(0, max(pokemon_votes_fam)*1.05, bounds=(0, max(pokemon_votes_fam)*1.05))
p_family.ygrid.grid_line_color = None

hover_family = HoverTool()
hover_family.tooltips = """
    <div>
        <h3>Family: @family</h3>
        <div><strong>Votes: </strong>@votes</div>
    </div>
"""
p_family.add_tools(hover_family)


# Create the "Votes in time" plot.
p_time = figure(plot_height=250, x_axis_type='datetime', y_axis_label="Votes")
p_time.vbar(df_time['timestamp'], bottom=0, top=df_time['vote'], width=0.9)
p_time.y_range = Range1d(0, max(df_time['vote'])*1.05, bounds=(0, max(df_time['vote'])*1.05))


# Create tabs.
tab1 = Panel(child=p_overall, title="Overall")
tab2 = Panel(child=p_generation, title="Generation")
tab3 = Panel(child=p_family, title="Family")
tab4 = Panel(child=p_time, title="Votes in time")
tabs = Tabs(tabs=[tab1, tab2, tab3, tab4])


def update(Pokemon):
    
    # Get Pokemon of interest parameters.
    pokemon_number = df_stats.index[df_stats.loc[:, 'name'] == Pokemon].tolist()[0]
    pokemon_generation = df_stats.loc[pokemon_number, 'generation']
    pokemon_family = df_stats.loc[pokemon_number, 'family']
    print("Generation " + str(pokemon_generation))
    
    # Update sprite.
    sprite.text = """<img src="https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/{}.png" alt="logo">""".format(pokemon_number)
    
    # Update overall.
    
    # Update generation.
    df_generation_ = df_sorted.query('generation=="' + str(pokemon_generation) + '"')
    source_generation_ = ColumnDataSource(df_generation_[['name', 'votes']])
    pokemon_names_gen_ = source_generation_.data['name']
    pokemon_votes_gen_ = source_generation_.data['votes']

    p_generation.x_range.bounds = (0, max(pokemon_votes_gen_)*1.05)
    p_generation.x_range.update(start=0, end=max(pokemon_votes_gen_)*1.05)
    p_generation.y_range.bounds = (0, len(pokemon_names_gen_))
    p_generation.y_range.factors = list(pokemon_names_gen_)
    
    r_generation.data_source.data.update(source_generation_.data)

    
    
    # Update family. 
    
    
    # Update votes in time.

    push_notebook()

l = layout(row(sprite, tabs), sizing_mode='stretch_width')
show(l, notebook_handle=True);

Loading BokehJS ...

In [401]:
interact(update, Pokemon=df_stats['name'].tolist());

interactive(children=(Dropdown(description='Pokemon', options=('Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander…